In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [ ]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [ ]:
a = 3.4
# lambdas = a/np.array([0.22,0.88]) 
lambdas = np.array([8,1.5])/0.8
# lambdas = a/np.array([0.35,0.8])


In [ ]:
# # folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250124"
# folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250116"
# project_name = "02_27_2025 Florescu LSU H5 Transmission eps"
# postprocess_results = []
# runtime_ps = 25e-12
# min_steps_per_lambda = 20
# cuts = [1]
# h5_bg = 1.42**2
# for direction in ["z"]: 
#     for f,filename in enumerate(os.listdir(folder_path)):
#      for cut in cuts:
#         if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
#             continue 
#         if filename in ["ak4_1000_eps_11p56_phih5_0p30_phiL_0p29.h5","ak4_1000_eps_12p96_phih5_0p30_phiL_0p29.h5","ak4_1000_eps_16p00_phih5_0p30_phiL_0p29.h5","ak4_1000_eps_2p3716_phih5_0p30_phiL_0p29.h5"]:
#            continue
           
#         if os.path.isfile(os.path.join(folder_path, filename)):
#             file=os.path.join(folder_path, filename)
#             structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
#                                             ,direction=direction, lambda_range=lambdas,
#                                             box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
#                                            scaling=1,shuoff_condtion=1e-20, verbose=True, 
#                                            monitors=["flux"], flux_monitor_position=24,cell_size_manual=50,
#                                            freqs=300, 
#                                            cut_condition=cut, source="planewave", absorbers=130, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg,
#                                            sim_bg=h5_bg

#                                            )
        
            
        
#         file_desc = rf"H:\phd stuff\tidy3d\data\{project_name}_perm_{structure_1.permittivity_value}\z_incidence\{structure_1.sim_name}.txt"
#         print(file_desc)
#         if os.path.exists(file_desc):
#             print("Exist!")
#         else:
#            print("Creating...")
#            structure_1.run_sim(run_free=False,load=False,add_ref=True,folder_description=rf"{project_name}_perm_{structure_1.permittivity_value}",monitor=False)

#         del structure_1

            
#         # structure_1.plot_sim_layout()
#         # raise(Exception)



In [ ]:
# folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250124"
folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250116"
project_name = "02_27_2025 Florescu LSU H5 Transmission eps"
postprocess_results = []
runtime_ps = 25e-12
min_steps_per_lambda = 20
cuts = [1]
h5_bg = None
for direction in ["z"]: 
    for f,filename in enumerate(os.listdir(folder_path)):
     for cut in cuts:
        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
            continue 

        if not filename in ["ak4_1000_eps_9p0_phih5_0p30_phiL_0p29.h5"]:
           continue

           
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=lambdas,
                                            box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                                           scaling=1,shuoff_condtion=1e-20, verbose=True, 
                                           monitors=["flux"], flux_monitor_position=24,cell_size_manual=50,
                                           freqs=300, 
                                           cut_condition=cut, source="planewave", absorbers=130, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg

                                           )
        
            
        
        file_desc = rf"H:\phd stuff\tidy3d\data\{project_name}_perm_{structure_1.permittivity_value}\z_incidence\{structure_1.sim_name}.txt"
        print(file_desc)
        if os.path.exists(file_desc):
            print("Exist!")
        else:
           print("Creating...")
        #    structure_1.plot_sim_layout()

           structure_1.run_sim(run_free=False,load=False,add_ref=True,folder_description=rf"{project_name}_perm_{structure_1.permittivity_value}",monitor=True)

        # del structure_1

            
        # raise(Exception)

